## Variation for 2-qubit case

Here we explore a different variation of the idea in the General_State_Discrimination notebook. Instead of mapping to the classifier qubits we apply a universal 2-qubit unitary and map it directly on the input qubits instead of seperate classifier qubits. This method in general can be extended for higher qubit cases as well. The main advantage is the significant decrease in number of free parameters (7, instead of 33) as compared to the other case.

In [ ]:
from qiskit import QuantumCircuit,QuantumRegister,ClassicalRegister
import numpy as np
from qiskit.circuit import ParameterVector
from qiskit import Aer
from qiskit.compiler import transpile

Below we create a universal 2-qubit unitary with 7 free parameters. The states 00,01 correspond to 1st family of staes, 10 to 2nd family of states and 11 is interpreted as an inconclusive signal, same as before.

In [17]:
from qiskit.circuit import ControlledGate
from qiskit.circuit.library import RYGate

def classifier_circuit():
    ## Initializing the circuit
    input_qubit = QuantumRegister(2)
    classifier_cbit = ClassicalRegister(2)
    qc = QuantumCircuit(input_qubit,classifier_cbit)
    
    ## All parametrized gates
    U1 = ParameterVector("U1",7)
    
    ## First gate: Arbitrary two qubit unitary
    qc.rz(U1[0],input_qubit[0])
    qc.rz(U1[1],input_qubit[1])
    qc.cx(input_qubit[1],input_qubit[0])
    qc.rz(U1[2],input_qubit[0])
    qc.ry(U1[3],input_qubit[1])
    qc.cx(input_qubit[0],input_qubit[1])
    qc.ry(U1[4],input_qubit[1])
    qc.cx(input_qubit[1],input_qubit[0])
    qc.rz(U1[5],input_qubit[0])
    qc.rz(U1[6],input_qubit[1])
    qc.barrier()


    backend = Aer.get_backend('aer_simulator')
    qc1 = transpile(qc,backend = backend)
    
    return qc1, [U1] 

Here we classify between these two family of states (one is pure, other is a mixed state):

1. $\psi_1 = a|00\rangle + \sqrt{1-a^2}|01\rangle$

2. $\rho_2 = \frac{1}{2}(|\psi_2^+\rangle\langle \psi_2^+| +|\psi_2^-\rangle\langle \psi_2^-|),\text{ where, }\psi_2 = b|01\rangle \pm \sqrt{1-b^2}|10\rangle$

Below function creates the states based on the label and corresponding parameter value

In [18]:
def input_circ(label,var):
    qc = QuantumCircuit(2)
    theta = 2 * np.arccos(var)   
    
    if label == 0 or label == 1:
        qc.rx(theta,1)
        
    elif label == 2:
        sign = np.random.choice([1,-1])
        qc.rx(sign * theta,1)
        qc.cx(1,0)
        qc.x(0)
        
    return qc  

In [19]:
## This cell contains functions for running the circuit and measuring loss 
## functions for each iteration

def qnn(qc,parameter_obs,parameter_vals,shots = 2048):
    
    """Returns counts from circuit based on input state and classifier circuit"""
    parameters = []
    p1 = 0
    p2 = 0
    for ind in range(len(parameter_obs)):
        p1 = p2
        p2 = p2 + len(parameter_obs[ind])
        parameters.append(parameter_vals[p1:p2])
        
    all_params = {}
    for i in range(len(parameter_obs)):
        all_params[parameter_obs[i]] = parameters[i]
    
    qc1 = qc.bind_parameters(all_params)
    
    qregist = qc1.qregs[0][:2]
    cregist = qc1.cregs[0]

    qc1.measure(qregist,cregist)
    
    backend = Aer.get_backend("aer_simulator")
    counts = backend.run(qc1,shots = shots).result().get_counts()
    
    return counts
    
def compute_loss_each(counts,y,alp_err,alp_inc,shots):
    
    """Returns the value of loss function for each data point in training data"""
    P_suc = 0
    P_err = 0
    P_inc = 0
    
    for states in counts:
        if (int(states,2) == 0 or int(states,2) == 1) and (y==0 or y==1) :
            P_suc += counts[states]/shots
        
        elif int(states,2) == 2 and y == 2:
            P_suc += counts[states]/shots
            
        elif int(states,2) == 3:
            P_inc += counts[states]/shots
        
        else:
            P_err += counts[states]/shots
    
    return [(1 - P_suc) +  alp_err * P_err + alp_inc * P_inc, P_suc,P_err,P_inc]


def compute_loss(parameter_vals, X, y,classifier_circ,parameter_obs,alp_err,alp_inc,shots = 2048):
    
    
    """Returns average loss function over the entire training dataset.
    Also prints the associated values of training for every 20 iterations till convergence"""
    
    global P
    P += 1

    each_loss = 0
    each_suc = 0
    each_err = 0
    each_inc = 0
    
    
    for ind in range(len(y)):
        total_circ = QuantumCircuit(2,2)
        input_qc = input_circ(y[ind],X[ind])
        
        total_circ.append(input_qc,[0,1])
                
        total_circ.append(classifier_circ,range(2),range(2))
        backend = Aer.get_backend("aer_simulator")
        qc1 = transpile(total_circ, backend=backend)
        
        counts = qnn(qc1,parameter_obs,parameter_vals,shots)
        loss_info = compute_loss_each(counts,y[ind],alp_err,alp_inc,shots)
        
        each_loss += loss_info[0]
        each_suc += loss_info[1]
        each_err += loss_info[2]
        each_inc += loss_info[3]
        
    out = each_loss/len(y)
    
    succ = each_suc/len(y)
    err = each_err/len(y)
    inc = each_inc/len(y)
    
    if P%20 == 0:
        print(f"Iterations = {P}, Function value = {out : .4f}, Success = {succ: .4f}, Error = {err:.4f}, Inconc. = {inc: .4f}")
     
    return out

In [20]:
## Function for testing success rate on test data

def test_succ(X,y,parameter_vals):
    """Measures the success over test dataset.
    Returns test success and inconclusive signal rate."""
    classifier,parameter_obs = classifier_circuit()
    
    succ = 0
    inc = 0
    
    for i in range(len(y)):
        input_circ_test = input_circ(y[i],X[i])
        
        total_circ = QuantumCircuit(2,2)
        total_circ.append(input_circ_test,[0,1])
                
        total_circ.append(classifier,range(2),range(2))
        backend = Aer.get_backend("aer_simulator")
        qc1 = transpile(total_circ, backend=backend)
        
        
        counts = qnn(qc1,parameter_obs,parameter_vals,shots=1024)
        
        maxim = 0
        
        for states in counts:
            if counts[states] > maxim:
                maxim = counts[states]
                maxim_state = states
                
        dec_maxim = int(maxim_state,2)
        if y[i] == 0 or y[i] == 1:
            if dec_maxim == 0 or dec_maxim == 1:
                succ += 1
        
        elif y[i] == 2:
            if dec_maxim == 2:
                succ += 1
        
        if dec_maxim == 3:
            inc += 1
     
    return succ/len(y),inc/len(y)
        
        

## $a\in[0,1], b\in[0,1]$

In [ ]:
## Functions to create dataset based on the above distribution
def create_data(data_size):
    data_val = []
    data_label = []
    
    for _ in range(data_size):
        data_label.append(np.random.choice([0,1,2],p = [0.25,0.25,0.5]))
        data_val.append(np.random.uniform(0,1))
        
    return data_val, data_label

### Trial values $\alpha_{err} = 0.2, \alpha_{inc} = 0.1$

In [41]:
from scipy.optimize import minimize

X,y = create_data(300)
alp_err,alp_inc = 0.2,0.1

P = 0

init_guess = np.random.uniform(low = 0, high = 2*np.pi,size = 7)
classifier_circ,parameter_obs = classifier_circuit()
res_qnn = minimize(compute_loss,init_guess,method = 'COBYLA',args = (X, y,classifier_circ,parameter_obs,alp_err,alp_inc))


Iterations = 20, Function value =  0.7001, Success =  0.3331, Error = 0.1654, Inconc. =  0.0014
Iterations = 40, Function value =  0.7006, Success =  0.3328, Error = 0.1666, Inconc. =  0.0006
Iterations = 60, Function value =  0.7013, Success =  0.3322, Error = 0.1671, Inconc. =  0.0006


In [43]:
X_test,y_test = create_data(600)

test_succ(X_test,y_test,res_qnn.x)

(0.6133333333333333, 0.0)

### $\alpha_{err} = 20, \alpha_{inc} = 10$

In [46]:
X,y = create_data(300)
alp_err,alp_inc = 20,10

P = 0

init_guess = np.random.uniform(low = 0, high = 2*np.pi,size = 7)
classifier_circ,parameter_obs = classifier_circuit()
res_qnn = minimize(compute_loss,init_guess,method = 'COBYLA',args = (X, y,classifier_circ,parameter_obs,alp_err,alp_inc))


Iterations = 20, Function value =  6.9306, Success =  0.6672, Error = 0.3270, Inconc. =  0.0058
Iterations = 40, Function value =  6.8473, Success =  0.6739, Error = 0.3260, Inconc. =  0.0001
Iterations = 60, Function value =  6.8557, Success =  0.6735, Error = 0.3264, Inconc. =  0.0000
Iterations = 80, Function value =  6.8439, Success =  0.6741, Error = 0.3259, Inconc. =  0.0000


In [48]:
X_test,y_test = create_data(600)

test_succ(X_test,y_test,res_qnn.x)

(0.65, 0.0)

## $\alpha_{err} = 20, \alpha_{inc} = 2$

In [49]:
X,y = create_data(300)

alp_err,alp_inc = 20,2
P = 0

init_guess = np.random.uniform(low = 0, high = 2*np.pi,size = 7)
classifier_circ,parameter_obs = classifier_circuit()
res_qnn = minimize(compute_loss,init_guess,method = 'COBYLA',args = (X, y,classifier_circ,parameter_obs,alp_err,alp_inc))


Iterations = 20, Function value =  6.3468, Success =  0.6851, Error = 0.3001, Inconc. =  0.0148
Iterations = 40, Function value =  6.3174, Success =  0.6851, Error = 0.2985, Inconc. =  0.0164
Iterations = 60, Function value =  6.3269, Success =  0.6847, Error = 0.2989, Inconc. =  0.0164


In [50]:
X_test,y_test = create_data(1000)

test_succ(X_test,y_test,res_qnn.x)

(0.656, 0.0)

Decreasing $\alpha_{inc}$ increases the accuracy and increases inconclusive rate on training data as expected. The change however is not as dramatic as expected from the paper.

## $a\approx 0.25,b \in [0,1]$

In [21]:
## Functions to create dataset based on the above distribution
def create_data(data_size):
    data_val = []
    data_label = []
    
    for _ in range(data_size):
        data_label.append(np.random.choice([0,1,2],p = [0.25,0.25,0.5]))
        if data_label[-1] == 0 or data_label[-1] == 1:
            data_val.append(np.random.uniform(0.249,0.251))
            
        elif data_label[-1] == 2:
            data_val.append(np.random.uniform(0,1))
    return data_val, data_label

In [22]:
X,y = create_data(300)
X_test,y_test = create_data(1000)

### $\alpha_{err} = 20, \alpha_{inc} = 10$

In [23]:
alp_err,alp_inc = 20,10
P = 0

init_guess = np.random.uniform(low = 0, high = 2*np.pi,size = 7)
classifier_circ,parameter_obs = classifier_circuit()
res_qnn = minimize(compute_loss,init_guess,method = 'COBYLA',args = (X, y,classifier_circ,parameter_obs,alp_err,alp_inc))

Iterations = 20, Function value =  6.9408, Success =  0.6557, Error = 0.3153, Inconc. =  0.0290
Iterations = 40, Function value =  6.9410, Success =  0.6555, Error = 0.3152, Inconc. =  0.0293
Iterations = 60, Function value =  6.9984, Success =  0.6529, Error = 0.3180, Inconc. =  0.0291


In [24]:
test_succ(X_test,y_test,res_qnn.x)

(0.626, 0.0)

### $\alpha_{err} = 20, \alpha_{inc} = 2$

In [67]:
alp_err,alp_inc = 20,2
P = 0

init_guess = np.random.uniform(low = 0, high = 2*np.pi,size = 7)
classifier_circ,parameter_obs = classifier_circuit()
res_qnn = minimize(compute_loss,init_guess,method = 'COBYLA',args = (X, y,classifier_circ,parameter_obs,alp_err,alp_inc))

Iterations = 20, Function value =  7.5562, Success =  0.6309, Error = 0.3583, Inconc. =  0.0108
Iterations = 40, Function value =  7.2766, Success =  0.6412, Error = 0.3445, Inconc. =  0.0143
Iterations = 60, Function value =  7.2783, Success =  0.6412, Error = 0.3446, Inconc. =  0.0142


In [68]:
test_succ(X_test,y_test,res_qnn.x)

(0.652, 0.0)

### $\alpha_{err} = 40, \alpha_{inc} = 2$

In [71]:
alp_err,alp_inc = 40,2
P = 0

init_guess = np.random.uniform(low = 0, high = 2*np.pi,size = 7)
classifier_circ,parameter_obs = classifier_circuit()
res_qnn = minimize(compute_loss,init_guess,method = 'COBYLA',args = (X, y,classifier_circ,parameter_obs,alp_err,alp_inc))

Iterations = 20, Function value =  14.3049, Success =  0.6240, Error = 0.3468, Inconc. =  0.0293
Iterations = 40, Function value =  14.0108, Success =  0.6310, Error = 0.3396, Inconc. =  0.0294
Iterations = 60, Function value =  14.3044, Success =  0.6237, Error = 0.3467, Inconc. =  0.0296


In [72]:
test_succ(X_test,y_test,res_qnn.x)

(0.654, 0.0)

Decreasing $\alpha_{inc}$ increases the accuracy and increases inconclusive rate on training data as expected. The change however is not as dramatic as expected from the paper.

## $a\in [0,1], b\approx \frac{1}{\sqrt{2}} $

In [7]:
## Functions to create dataset based on the above distribution
def create_data(data_size):
    data_val = []
    data_label = []
    
    for _ in range(data_size):
        data_label.append(np.random.choice([0,1,2],p = [0.25,0.25,0.5]))
        if data_label[-1] == 0 or data_label[-1] == 1:
            data_val.append(np.random.uniform(0,1))
            
        elif data_label[-1] == 2:
            data_val.append(np.random.uniform(1/np.sqrt(2)-0.001,1/np.sqrt(2)+0.001))
    return data_val, data_label

In [8]:
X,y = create_data(300)
X_test,y_test = create_data(1000)

### $\alpha_{err} = 20, \alpha_{inc} = 10$

In [26]:
from scipy.optimize import minimize

alp_err,alp_inc = 20,10
P = 0

init_guess = np.random.uniform(low = 0, high = 2*np.pi,size = 7)
classifier_circ,parameter_obs = classifier_circuit()
res_qnn = minimize(compute_loss,init_guess,method = 'COBYLA',args = (X, y,classifier_circ,parameter_obs,alp_err,alp_inc))

Iterations = 20, Function value =  6.6350, Success =  0.6838, Error = 0.3156, Inconc. =  0.0006
Iterations = 40, Function value =  6.6484, Success =  0.6834, Error = 0.3165, Inconc. =  0.0001
Iterations = 60, Function value =  6.6193, Success =  0.6847, Error = 0.3151, Inconc. =  0.0001


In [28]:
test_succ(X_test,y_test,res_qnn.x)

(0.62, 0.0)

### $\alpha_{err} = 20, \alpha_{inc} = 2$

In [12]:
alp_err,alp_inc = 20,2
P = 0

init_guess = np.random.uniform(low = 0, high = 2*np.pi,size = 7)
classifier_circ,parameter_obs = classifier_circuit()
res_qnn = minimize(compute_loss,init_guess,method = 'COBYLA',args = (X, y,classifier_circ,parameter_obs,alp_err,alp_inc))

Iterations = 20, Function value =  5.2189, Success =  0.6716, Error = 0.2352, Inconc. =  0.0932
Iterations = 40, Function value =  5.1223, Success =  0.6798, Error = 0.2312, Inconc. =  0.0891
Iterations = 60, Function value =  5.1458, Success =  0.6786, Error = 0.2323, Inconc. =  0.0891


In [13]:
test_succ(X_test,y_test,res_qnn.x)

(0.773, 0.003)

### $\alpha_{err} = 40, \alpha_{inc} = 2$

In [14]:
alp_err,alp_inc = 40,2
P = 0

init_guess = np.random.uniform(low = 0, high = 2*np.pi,size = 7)
classifier_circ,parameter_obs = classifier_circuit()
res_qnn = minimize(compute_loss,init_guess,method = 'COBYLA',args = (X, y,classifier_circ,parameter_obs,alp_err,alp_inc))

Iterations = 20, Function value =  9.8725, Success =  0.7443, Error = 0.2396, Inconc. =  0.0161
Iterations = 40, Function value =  9.8584, Success =  0.7474, Error = 0.2395, Inconc. =  0.0131
Iterations = 60, Function value =  9.9600, Success =  0.7449, Error = 0.2420, Inconc. =  0.0131


In [15]:
test_succ(X_test,y_test,res_qnn.x)

(0.762, 0.0)

Decreasing $\alpha_{inc}$ increases the accuracy and increases inconclusive rate on training data as expected. The change however is not as dramatic as expected from the paper.

### Conclusion

We notice a very similar accuracy in both the circuit mentioned in the paper and the variation explored here. This demonstrates that the classification capability is very similar at significantly lower number of free parameters. This method should be preffered when discriminating between states, when the measurement does not need to preserve the state